In [ ]:
"""
Purpose: To check on the status of the soma finder


"""

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

In [2]:
import minfig
du = reload(du)
du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-22 09:43:09,945 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-22 09:43:09,946 - settings - Setting database.user to celiib
INFO - 2020-11-22 09:43:09,947 - settings - Setting database.password to newceliipass
INFO - 2020-11-22 09:43:09,948 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-22 09:43:09,949 - settings - Setting database.user to celiib
INFO - 2020-11-22 09:43:09,950 - settings - Setting database.password to newceliipass
INFO - 2020-11-22 09:43:09,955 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/de

Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons
Connecting celiib@at-database.ad.bcm.edu:3306


# Inspecting the Decimation

In [ ]:
(schema.jobs & "table_name='__decimation'").delete()

In [ ]:
len(minnie.Mesh() & minnie.SegToDecimateFromNuclei())

In [ ]:
decimation_version = 0
decimation_ratio = 0.25
len(minnie.Decimation().proj(decimation_version='version') & "decimation_version=" + str(decimation_version) &
                   f"decimation_ratio={decimation_ratio}" & minnie.SegToDecimateFromNuclei() ) - 114201

# Inspecting the Soma Extraction

In [ ]:
(schema.jobs & 'table_name="__baylor_segment_centroid"').delete()

In [ ]:
decimation_version = 0
decimation_ratio = 0.25
verts_min = 10000

key_source =  ((minnie.Decimation & f"n_vertices > {verts_min}").proj(decimation_version='version') & 
                        "decimation_version=" + str(decimation_version) &
                   f"decimation_ratio={decimation_ratio}" & minnie.SegToDecimateFromNuclei())
len(key_source)

In [ ]:
len(key_source) - len(key_source.proj() & (minnie.BaylorSegmentCentroid()).proj())

In [ ]:
len(minnie.BaylorSegmentCentroid() & "multiplicity>0")/len(minnie.BaylorSegmentCentroid())*len(key_source)

# Chccking the current progress of Soma Extraction

In [ ]:
my_key = (minnie.BaylorSegmentCentroid() & "segment_id=864691134026921572").fetch1()
my_key

In [ ]:
"""



"""

In [ ]:
minnie.Decimation.describe()

In [ ]:
minnie.BaylorSegmentCentroidExternal & "segment_id=864691134026921572"

In [ ]:
len(minnie.BaylorSegmentCentroid())

In [ ]:
len(minnie.BaylorSegmentCentroidExternal())

In [ ]:
p = Path('/mnt/dj-stor01/platinum/minnie65/02/somas/')

In [ ]:
for f in p.iterdir():
    f.unlink()

In [ ]:
no_soma_seg_id = (minnie.Decimation() & (minnie.BaylorSegmentCentroidExternal & "multiplicity=0").proj() & "n_vertices>50000").fetch("segment_id")

In [ ]:
curr_seg_id = no_soma_seg_id[200]

In [ ]:
curr_seg_id

In [ ]:

new_mesh = (minnie.Decimation() & key).fetch1("mesh")
current_mesh_verts,current_mesh_faces = new_mesh.vertices,new_mesh.faces


In [ ]:
du.plot_decimated_mesh_with_somas(curr_seg_id)

In [ ]:
curr_mesh = du.fetch_segment_id_mesh(curr_seg_id)

In [ ]:
import soma_extraction_utils as sm
soma_data = sm.extract_soma_center(segment_id=curr_seg_id,
                      current_mesh_verts=current_mesh_verts,
                      current_mesh_faces=current_mesh_faces)

In [ ]:
nviz.plot_objects(meshes=soma_data[0])

In [4]:
import soma_extraction_utils as sm
key = {'segment_id': 864691134939968355, 'decimation_version': 0, 'decimation_ratio': 0.25}
new_mesh = (minnie.Decimation() & key).fetch1("mesh")
current_mesh_verts,current_mesh_faces = new_mesh.vertices,new_mesh.faces

segment_id = key["segment_id"]

(total_soma_list, 
 run_time, 
 total_soma_list_sdf) = sm.extract_soma_center(
                    segment_id,
                    current_mesh_verts,
                    current_mesh_faces,
)

Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 5000.0 
large_mesh_threshold_inner = 5000.0 
soma_size_threshold = 937.5 
soma_size_threshold_max = 12000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
xvfb-run -n 8818 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Soma_Extraction/temp/neuron_78575.off -o /notebooks/Platinum_Datajoint/Soma_Extraction/temp/neuron_78575_fill_holes.off -s /notebooks/Platinum_Datajoint/Soma_Extraction/temp/fill_holes_241800.mls

---- meshlab output -----
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'
Current Plugins Dir is: /meshlab/src/distrib/plugins 
Error While parsing the XML filter plugin descriptors: We are trying to load a xml file that does not correspond to any dll or javascript code; please delete all the spurious xml files
Error While parsing the XML filter plugin descriptors: We are trying to load a xml file that does not correspond to any dll or 

WARNING - 2020-11-22 09:43:52,551 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:52,552 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:52,569 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:52,569 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:52,714 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:52,716 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:52,746 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:52,784 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:52,786 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:52,831 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:52,832 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:52,840 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:52,849 - base - face_normals all zero, ignoring!
WARNING - 20

WARNING - 2020-11-22 09:43:54,263 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:54,264 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:54,267 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:54,268 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:54,270 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:54,271 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:54,285 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:54,288 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:54,320 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:54,326 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:54,359 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:54,359 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:43:54,362 - base - face_normals all zero, ignoring!
WARNING - 20

Removing the following inside neurons: [<trimesh.Trimesh(vertices.shape=(2853, 3), faces.shape=(6331, 3))>, <trimesh.Trimesh(vertices.shape=(2721, 3), faces.shape=(6525, 3))>, <trimesh.Trimesh(vertices.shape=(1954, 3), faces.shape=(4622, 3))>, <trimesh.Trimesh(vertices.shape=(1723, 3), faces.shape=(2712, 3))>, <trimesh.Trimesh(vertices.shape=(1686, 3), faces.shape=(2629, 3))>, <trimesh.Trimesh(vertices.shape=(1623, 3), faces.shape=(2497, 3))>, <trimesh.Trimesh(vertices.shape=(1613, 3), faces.shape=(2477, 3))>, <trimesh.Trimesh(vertices.shape=(1502, 3), faces.shape=(3854, 3))>, <trimesh.Trimesh(vertices.shape=(1178, 3), faces.shape=(2712, 3))>, <trimesh.Trimesh(vertices.shape=(1122, 3), faces.shape=(2583, 3))>, <trimesh.Trimesh(vertices.shape=(998, 3), faces.shape=(2400, 3))>, <trimesh.Trimesh(vertices.shape=(894, 3), faces.shape=(2047, 3))>, <trimesh.Trimesh(vertices.shape=(824, 3), faces.shape=(1942, 3))>, <trimesh.Trimesh(vertices.shape=(757, 3), faces.shape=(1117, 3))>, <trimesh.Tri

WARNING - 2020-11-22 09:44:02,247 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:02,363 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:02,364 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:02,463 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:02,466 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:02,471 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:02,482 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:02,492 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:02,498 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:02,499 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:02,510 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:02,511 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:02,515 - base - face_normals all zero, ignoring!
WARNING - 20

...

Using Poisson Surface Reconstruction for watertightness in soma_volume_ratio
xvfb-run -n 9202 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Soma_Extraction/Poisson_temp/neuron_359113.off -o /notebooks/Platinum_Datajoint/Soma_Extraction/Poisson_temp/neuron_359113_poisson.off -s /notebooks/Platinum_Datajoint/Soma_Extraction/Poisson_temp/poisson_755056.mls
removed temporary input file: /notebooks/Platinum_Datajoint/Soma_Extraction/Poisson_temp/neuron_359113.off
removed temporary output file: /notebooks/Platinum_Datajoint/Soma_Extraction/Poisson_temp/neuron_359113_poisson.off
mesh.is_watertight = True
/notebooks/Platinum_Datajoint/Soma_Extraction/Poisson_temp/poisson_755056.mls is being deleted....
Inside sphere validater: ratio_val = 186.17428406381703
--->This soma mesh was not added because it did not pass the sphere validation:
 soma_mesh = <trimesh.Trimesh(vertices.shape=(874, 3), faces.shape=(1453, 3))>, curr_side_len_check = True, curr_volume_check = F

WARNING - 2020-11-22 09:44:38,737 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:38,845 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:38,846 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:38,945 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:38,949 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:38,956 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:38,966 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:38,977 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:38,982 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:38,983 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:38,994 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:38,995 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 09:44:39,006 - base - face_normals all zero, ignoring!
WARNING - 20

# total split meshes = 30
viable_meshes = [0]
There were 29 pieces found after size threshold
# of soma containing seperate meshes = 1
meshes with somas = {0: [0]}


----Working on soma-containing mesh piece 0----
current_soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(4369, 3), faces.shape=(3711, 3))>]
current_mesh = <trimesh.Trimesh(vertices.shape=(97900, 3), faces.shape=(195455, 3))>

inside Soma subtraction
mesh pieces in subtact soma BEFORE the filtering inside pieces = [<trimesh.Trimesh(vertices.shape=(24796, 3), faces.shape=(49444, 3))>, <trimesh.Trimesh(vertices.shape=(9482, 3), faces.shape=(18863, 3))>, <trimesh.Trimesh(vertices.shape=(8490, 3), faces.shape=(16153, 3))>, <trimesh.Trimesh(vertices.shape=(2965, 3), faces.shape=(5935, 3))>, <trimesh.Trimesh(vertices.shape=(2751, 3), faces.shape=(5459, 3))>, <trimesh.Trimesh(vertices.shape=(2209, 3), faces.shape=(4399, 3))>, <trimesh.Trimesh(vertices.shape=(1892, 3), faces.shape=(3779, 3))>, <trimesh.Trimesh(vertices.shape=(138

/meshAfterParty/trimesh_utils.py:519: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ordered_comp_indices = np.array([k.astype("int") for k in ordered_components])


Using Poisson Surface Reconstruction for watertightness in soma_volume_ratio
xvfb-run -n 1339 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Soma_Extraction/Poisson_temp/neuron_178008.off -o /notebooks/Platinum_Datajoint/Soma_Extraction/Poisson_temp/neuron_178008_poisson.off -s /notebooks/Platinum_Datajoint/Soma_Extraction/Poisson_temp/poisson_256982.mls
removed temporary input file: /notebooks/Platinum_Datajoint/Soma_Extraction/Poisson_temp/neuron_178008.off
removed temporary output file: /notebooks/Platinum_Datajoint/Soma_Extraction/Poisson_temp/neuron_178008_poisson.off
mesh.is_watertight = True
/notebooks/Platinum_Datajoint/Soma_Extraction/Poisson_temp/poisson_256982.mls is being deleted....
Inside sphere validater: ratio_val = 52.96064129556593
--->This soma mesh was not added because it did not pass the sphere validation:
 soma_mesh = <trimesh.Trimesh(vertices.shape=(31307, 3), faces.shape=(61455, 3))>, curr_side_len_check = False, curr_volume_check 

In [5]:
total_soma_list

[]

# Debug the Soma Finding

In [29]:
from soma_extraction_utils import *

key = {'segment_id': 864691134939968355, 'decimation_version': 0, 'decimation_ratio': 0.25}
new_mesh = (minnie.Decimation() & key).fetch1("mesh")
current_mesh_verts,current_mesh_faces = new_mesh.vertices,new_mesh.faces

outer_decimation_ratio= 0.25
large_mesh_threshold = 20000#60000,
large_mesh_threshold_inner = 20000
soma_width_threshold = 0.32
soma_size_threshold = 15000
inner_decimation_ratio = 0.25
volume_mulitplier=8
#side_length_ratio_threshold=3
side_length_ratio_threshold=6
soma_size_threshold_max=192000 #this puts at 12000 once decimated, another possible is 256000
delete_files=True
backtrack_soma_mesh_to_original=True #should either be None or 
boundary_vertices_threshold=None#700 the previous threshold used
poisson_backtrack_distance_threshold=None#1500 the previous threshold used
close_holes=False

#------- 11/12 Additions --------------- #

#these arguments are for removing inside pieces
remove_inside_pieces = True
size_threshold_to_remove=1000 #size accounting for the decimation


pymeshfix_clean=False
check_holes_before_pymeshfix=False
second_poisson=False
segmentation_at_end=True
last_size_threshold = 2000#1300,

largest_hole_threshold = 17000


global_start_time = time.time()

#Adjusting the thresholds based on the decimations
large_mesh_threshold = large_mesh_threshold*outer_decimation_ratio
large_mesh_threshold_inner = large_mesh_threshold_inner*outer_decimation_ratio
soma_size_threshold = soma_size_threshold*outer_decimation_ratio
soma_size_threshold_max = soma_size_threshold_max*outer_decimation_ratio

#adjusting for inner decimation
soma_size_threshold = soma_size_threshold*inner_decimation_ratio
soma_size_threshold_max = soma_size_threshold_max*inner_decimation_ratio
print(f"Current Arguments Using (adjusted for decimation):\n large_mesh_threshold= {large_mesh_threshold}"
             f" \nlarge_mesh_threshold_inner = {large_mesh_threshold_inner}"
              f" \nsoma_size_threshold = {soma_size_threshold}"
             f" \nsoma_size_threshold_max = {soma_size_threshold_max}"
             f"\nouter_decimation_ratio = {outer_decimation_ratio}"
             f"\ninner_decimation_ratio = {inner_decimation_ratio}")


# ------------------------------


temp_folder = f"./{segment_id}"
temp_object = Path(temp_folder)
#make the temp folder if it doesn't exist
temp_object.mkdir(parents=True,exist_ok=True)

#making the decimation and poisson objections
Dec_outer = meshlab.Decimator(outer_decimation_ratio,temp_folder,overwrite=True)
Dec_inner = meshlab.Decimator(inner_decimation_ratio,temp_folder,overwrite=True)
Poisson_obj = meshlab.Poisson(temp_folder,overwrite=True)


recov_orig_mesh = trimesh.Trimesh(vertices=current_mesh_verts,faces=current_mesh_faces)
recov_orig_mesh_no_interior = tu.remove_mesh_interior(recov_orig_mesh)


#Step 1: Decimate the Mesh and then split into the seperate pieces
new_mesh,output_obj = Dec_outer(vertices=recov_orig_mesh_no_interior.vertices,
         faces=recov_orig_mesh_no_interior.faces,
         segment_id=segment_id,
         return_mesh=True,
         delete_temp_files=False)

# if remove_inside_pieces:
#     print("removing mesh interior after decimation")
#     new_mesh = tu.remove_mesh_interior(new_mesh,size_threshold_to_remove=size_threshold_to_remove)

#preforming the splits of the decimated mesh

mesh_splits = new_mesh.split(only_watertight=False)

#get the largest mesh
mesh_lengths = np.array([len(split.faces) for split in mesh_splits])


total_mesh_split_lengths = [len(k.faces) for k in mesh_splits]
ordered_mesh_splits = mesh_splits[np.flip(np.argsort(total_mesh_split_lengths))]
list_of_largest_mesh = [k for k in ordered_mesh_splits if len(k.faces) > large_mesh_threshold]

print(f"Total found significant pieces before Poisson = {list_of_largest_mesh}")

#if no significant pieces were found then will use smaller threshold
if len(list_of_largest_mesh)<=0:
    print(f"Using smaller large_mesh_threshold because no significant pieces found with {large_mesh_threshold}")
    list_of_largest_mesh = [k for k in ordered_mesh_splits if len(k.faces) > large_mesh_threshold/2]

total_soma_list = []
total_classifier_list = []
total_poisson_list = []
total_soma_list_sdf = []



#start iterating through where go through all pieces before the poisson reconstruction
no_somas_found_in_big_loop = 0
for i,largest_mesh in enumerate(list_of_largest_mesh):
    print(f"----- working on large mesh #{i}: {largest_mesh}")

    if remove_inside_pieces:
        print("remove_inside_pieces requested ")
        largest_mesh = tu.remove_mesh_interior(largest_mesh,size_threshold_to_remove=size_threshold_to_remove)


    if pymeshfix_clean:
        print("Requested pymeshfix_clean")
        """
        Don't have to check if manifold anymore actually just have to plug the holes
        """
        hole_groups = tu.find_border_face_groups(largest_mesh)
        if len(hole_groups) > 0:
            largest_mesh_filled_holes = tu.fill_holes(largest_mesh,max_hole_size = 10000)
        else:
            largest_mesh_filled_holes = largest_mesh

        if check_holes_before_pymeshfix:
            hole_groups = tu.find_border_face_groups(largest_mesh_filled_holes)
        else:
            print("Not checking if there are still existing holes before pymeshfix")
            hole_groups = []

        if len(hole_groups) > 0:
            #segmentation_at_end = False
            print(f"*** COULD NOT FILL HOLES WITH MAX SIZE OF {np.max([len(k) for k in hole_groups])} so not applying pymeshfix and segmentation_at_end = {segmentation_at_end}")

#                 tu.write_neuron_off(largest_mesh_filled_holes,"largest_mesh_filled_holes")
#                 raise Exception()
        else:
            print("Applying pymeshfix_clean because no more holes")
            largest_mesh = tu.pymeshfix_clean(largest_mesh_filled_holes,verbose=True)

    if second_poisson:
        print("Applying second poisson run")
        current_neuron_poisson = tu.poisson_surface_reconstruction(largest_mesh)
        largest_mesh = tu.split_significant_pieces(current_neuron_poisson)[0]

    somas_found_in_big_loop = False

    largest_file_name = str(output_obj.stem) + "_largest_piece.off"
    pre_largest_mesh_path = temp_object / Path(str(output_obj.stem) + "_largest_piece.off")
    pre_largest_mesh_path = pre_largest_mesh_path.absolute()
    print(f"pre_largest_mesh_path = {pre_largest_mesh_path}")
    # ******* This ERRORED AND CALLED OUR NERUON NONE: 77697401493989254 *********
    new_mesh_inner,poisson_file_obj = Poisson_obj(vertices=largest_mesh.vertices,
               faces=largest_mesh.faces,
               return_mesh=True,
               mesh_filename=largest_file_name,
               delete_temp_files=False)


    #splitting the Poisson into the largest pieces and ordering them
    mesh_splits_inner = new_mesh_inner.split(only_watertight=False)
    total_mesh_split_lengths_inner = [len(k.faces) for k in mesh_splits_inner]
    ordered_mesh_splits_inner = mesh_splits_inner[np.flip(np.argsort(total_mesh_split_lengths_inner))]

    list_of_largest_mesh_inner = [k for k in ordered_mesh_splits_inner if len(k.faces) > large_mesh_threshold_inner]
    print(f"Total found significant pieces AFTER Poisson = {list_of_largest_mesh_inner}")

    n_failed_inner_soma_loops = 0
    for j, largest_mesh_inner in enumerate(list_of_largest_mesh_inner):
        to_add_list = []
        to_add_list_sdf = []

        print(f"----- working on mesh after poisson #{j}: {largest_mesh_inner}")

        largest_mesh_path_inner = str(poisson_file_obj.stem) + "_largest_inner.off"

        #Decimate the inner poisson piece
        largest_mesh_path_inner_decimated,output_obj_inner = Dec_inner(
                            vertices=largest_mesh_inner.vertices,
                             faces=largest_mesh_inner.faces,
                            mesh_filename=largest_mesh_path_inner,
                             return_mesh=True,
                             delete_temp_files=False)

        dec_splits = tu.split_significant_pieces(largest_mesh_path_inner_decimated,significance_threshold=15)
        print(f"\n-------Splits after inner decimation len = {len(dec_splits)}--------\n")
        
        

        if len(dec_splits) == 0:
            print("There were no signifcant splits after inner decimation")
            n_failed_inner_soma_loops += 1
        else:
            print(f"done exporting decimated mesh: {largest_mesh_path_inner}")

            largest_mesh_path_inner_decimated_clean = dec_splits[0]
            print(f"largest_mesh_path_inner_decimated_clean = {largest_mesh_path_inner_decimated_clean}\n")

            faces = np.array(largest_mesh_path_inner_decimated_clean.faces)
            verts = np.array(largest_mesh_path_inner_decimated_clean.vertices)

            # may need to do some processing


            segment_id_new = int(str(segment_id) + f"{i}{j}")
            #print(f"Before the classifier the pymeshfix_clean = {pymeshfix_clean}")
            verts_labels, faces_labels, soma_value,classifier = wcda.extract_branches_whole_neuron(
                                    import_Off_Flag=False,
                                    segment_id=segment_id_new,
                                    vertices=verts,
                                     triangles=faces,
                                    pymeshfix_Flag=False,
                                     import_CGAL_Flag=False,
                                     return_Only_Labels=True,
                                     clusters=3,
                                     smoothness=0.2,
                                    soma_only=True,
                                    return_classifier = True
                                    )
            print(f"soma_sdf_value = {soma_value}")

            total_classifier_list.append(classifier)
            #total_poisson_list.append(largest_mesh_path_inner_decimated)

            # Save all of the portions that resemble a soma
            median_values = np.array([v["median"] for k,v in classifier.sdf_final_dict.items()])
            segmentation = np.array([k for k,v in classifier.sdf_final_dict.items()])

            #order the compartments by greatest to smallest
            sorted_medians = np.flip(np.argsort(median_values))
            print(f"segmentation[sorted_medians],median_values[sorted_medians] = {(segmentation[sorted_medians],median_values[sorted_medians])}")
            print(f"Sizes = {[classifier.sdf_final_dict[g]['n_faces'] for g in segmentation[sorted_medians]]}")
            print(f"soma_size_threshold = {soma_size_threshold}")
            print(f"soma_size_threshold_max={soma_size_threshold_max}")

            valid_soma_segments_width = [g for g,h in zip(segmentation[sorted_medians],median_values[sorted_medians]) if ((h > soma_width_threshold)
                                                                and (classifier.sdf_final_dict[g]["n_faces"] > soma_size_threshold)
                                                                and (classifier.sdf_final_dict[g]["n_faces"] < soma_size_threshold_max))]
            valid_soma_segments_sdf = [h for g,h in zip(segmentation[sorted_medians],median_values[sorted_medians]) if ((h > soma_width_threshold)
                                                                and (classifier.sdf_final_dict[g]["n_faces"] > soma_size_threshold)
                                                                and (classifier.sdf_final_dict[g]["n_faces"] < soma_size_threshold_max))]

            print("valid_soma_segments_width")
            

            if len(valid_soma_segments_width) > 0:
                print(f"      ------ Found {len(valid_soma_segments_width)} viable somas: {valid_soma_segments_width}")
                somas_found_in_big_loop = True
                #get the meshes only if signfiicant length
                labels_list = classifier.labels_list

                for v,sdf in zip(valid_soma_segments_width,valid_soma_segments_sdf):
                    submesh_face_list = np.where(classifier.labels_list == v)[0]
                    soma_mesh = largest_mesh_path_inner_decimated_clean.submesh([submesh_face_list],append=True)

                    # ---------- No longer doing the extra checks in here --------- #


                    curr_side_len_check = side_length_check(soma_mesh,side_length_ratio_threshold)
                    curr_volume_check = soma_volume_check(soma_mesh,volume_mulitplier)
                    if curr_side_len_check and curr_volume_check:
                        to_add_list.append(soma_mesh)
                        to_add_list_sdf.append(sdf)

                    else:
                        print(f"--->This soma mesh was not added because it did not pass the sphere validation:\n "
                             f"soma_mesh = {soma_mesh}, curr_side_len_check = {curr_side_len_check}, curr_volume_check = {curr_volume_check}")
                        continue

                n_failed_inner_soma_loops = 0

            else:
                n_failed_inner_soma_loops += 1

        total_soma_list_sdf += to_add_list_sdf
        total_soma_list += to_add_list

        # --------------- KEEP TRACK IF FAILED TO FIND SOMA (IF TOO MANY FAILS THEN BREAK)
        if n_failed_inner_soma_loops >= 2:
            print("breaking inner loop because 2 soma fails in a row")
            break


    # --------------- KEEP TRACK IF FAILED TO FIND SOMA (IF TOO MANY FAILS THEN BREAK)
    if somas_found_in_big_loop == False:
        no_somas_found_in_big_loop += 1
        if no_somas_found_in_big_loop >= 2:
            print("breaking because 2 fails in a row in big loop")
            break

    else:
        no_somas_found_in_big_loop = 0





""" IF THERE ARE MULTIPLE SOMAS THAT ARE WITHIN A CERTAIN DISTANCE OF EACH OTHER THEN JUST COMBINE THEM INTO ONE"""
pairings = []
for y,soma_1 in enumerate(total_soma_list):
    for z,soma_2 in enumerate(total_soma_list):
        if y<z:
            mesh_tree = KDTree(soma_1.vertices)
            distances,closest_node = mesh_tree.query(soma_2.vertices)

            if np.min(distances) < 4000:
                pairings.append([y,z])


#creating the combined meshes from the list
total_soma_list_revised = []
total_soma_list_revised_sdf = []
if len(pairings) > 0:
    """
    Pseudocode: 
    Use a network function to find components

    """


    import networkx as nx
    new_graph = nx.Graph()
    new_graph.add_edges_from(pairings)
    grouped_somas = list(nx.connected_components(new_graph))

    somas_being_combined = []
    print(f"There were soma pairings: Connected components in = {grouped_somas} ")
    for comp in grouped_somas:
        comp = list(comp)
        somas_being_combined += list(comp)
        current_mesh = total_soma_list[comp[0]]
        for i in range(1,len(comp)):
            current_mesh += total_soma_list[comp[i]] #just combining the actual meshes

        total_soma_list_revised.append(current_mesh)
        #where can average all of the sdf values
        total_soma_list_revised_sdf.append(np.min(np.array(total_soma_list_sdf)[comp]))

    #add those that weren't combined to total_soma_list_revised
    leftover_somas = [total_soma_list[k] for k in range(0,len(total_soma_list)) if k not in somas_being_combined]
    leftover_somas_sdfs = [total_soma_list_sdf[k] for k in range(0,len(total_soma_list)) if k not in somas_being_combined]
    if len(leftover_somas) > 0:
        total_soma_list_revised += leftover_somas
        total_soma_list_revised_sdf += leftover_somas_sdfs

    print(f"Final total_soma_list_revised = {total_soma_list_revised}")
    print(f"Final total_soma_list_revised_sdf = {total_soma_list_revised_sdf}")


if len(total_soma_list_revised) == 0:
    total_soma_list_revised = total_soma_list
    total_soma_list_revised_sdf = total_soma_list_sdf

run_time = time.time() - global_start_time

print(f"\n\n\n Total time for run = {time.time() - global_start_time}")
print(f"Before Filtering the number of somas found = {len(total_soma_list_revised)}")

#     import system_utils as su
#     su.compressed_pickle(total_soma_list_revised,"total_soma_list_revised")
#     su.compressed_pickle(new_mesh,"original_mesh")

#need to erase all of the temporary files ******
#import shutil
#shutil.rmtree(directory)

"""
Running the extra tests that depend on
- border vertices
- how well the poisson matches the backtracked soma to the real mesh
- other size checks

"""
filtered_soma_list = []
filtered_soma_list_sdf = []
for soma_mesh,curr_soma_sdf in zip(total_soma_list_revised,total_soma_list_revised_sdf):
    if backtrack_soma_mesh_to_original:
        print("Performing Soma Mesh Backtracking to original mesh")
        soma_mesh_poisson = deepcopy(soma_mesh)
        try:
            #print("About to find original mesh")
            soma_mesh = original_mesh_soma(
                                            mesh = recov_orig_mesh_no_interior,
                                            soma_meshes=[soma_mesh_poisson],
                                            sig_th_initial_split=15)[0]
        except:
            import traceback
            traceback.print_exc()
            print("--->This soma mesh was not added because Was not able to backtrack soma to mesh")
            continue
        else:
            if soma_mesh is None:
                print("--->This soma mesh was not added because Was not able to backtrack soma to mesh")
                continue




        print(f"poisson_backtrack_distance_threshold = {poisson_backtrack_distance_threshold}")
        #do the check that tests if there is a max distance between poisson and backtrack:
        if not poisson_backtrack_distance_threshold is None and poisson_backtrack_distance_threshold > 0:

            #soma_mesh.export("soma_mesh.off")
            if close_holes: 
                print("Using the close holes feature")
                fill_hole_obj = meshlab.FillHoles(max_hole_size=2000,
                                                 self_itersect_faces=False)

                soma_mesh_filled_holes,output_subprocess_obj = fill_hole_obj(   
                                                    vertices=soma_mesh.vertices,
                                                     faces=soma_mesh.faces,
                                                     return_mesh=True,
                                                     delete_temp_files=True,
                                                    )
            else:
                soma_mesh_filled_holes = soma_mesh


            #soma_mesh_filled_holes.export("soma_mesh_filled_holes.off")



            print("APPLYING poisson_backtrack_distance_threshold CHECKS")
            mesh_1 = soma_mesh_filled_holes
            mesh_2 = soma_mesh_poisson

            poisson_max_distance = tu.max_distance_betwee_mesh_vertices(mesh_1,mesh_2,
                                                              verbose=True)
            print(f"poisson_max_distance = {poisson_max_distance}")
            if poisson_max_distance > poisson_backtrack_distance_threshold:
                print(f"--->This soma mesh was not added because it did not pass the poisson_backtrack_distance check:\n"
                  f" poisson_max_distance = {poisson_max_distance}")
                continue

    if len(soma_mesh.faces) < 5:
        print(f"--> soma had very few faces ({soma_mesh}) so continuing")
        continue

    #do the boundary check:
    if not boundary_vertices_threshold is None:
        print("USING boundary_vertices_threshold CHECK")
        soma_boundary_groups_sizes = np.array([len(k) for k in tu.find_border_face_groups(soma_mesh)])
        print(f"soma_boundary_groups_sizes = {soma_boundary_groups_sizes}")
        large_boundary_groups = soma_boundary_groups_sizes[soma_boundary_groups_sizes>boundary_vertices_threshold]
        print(f"large_boundary_groups = {large_boundary_groups} with boundary_vertices_threshold = {boundary_vertices_threshold}")
        if len(large_boundary_groups)>0:
            print(f"--->This soma mesh was not added because it did not pass the boundary vertices validation:\n"
                  f" large_boundary_groups = {large_boundary_groups}")
            continue

    curr_side_len_check = side_length_check(soma_mesh,side_length_ratio_threshold)
    curr_volume_check = soma_volume_check(soma_mesh,volume_mulitplier)
    if (not curr_side_len_check) or (not curr_volume_check):
        print(f"--->This soma mesh was not added because it did not pass the sphere validation:\n "
             f"soma_mesh = {soma_mesh}, curr_side_len_check = {curr_side_len_check}, curr_volume_check = {curr_volume_check}")
        continue

    #tu.write_neuron_off(soma_mesh_poisson,"original_poisson.off")
    #If made it through all the checks then add to final list
    filtered_soma_list.append(soma_mesh)
    filtered_soma_list_sdf.append(curr_soma_sdf)


"""
Need to delete all files in the temp folder *****
"""

if delete_files:
    #now erase all of the files used
    from shutil import rmtree

    #remove the directory with the meshes
    rmtree(str(temp_object.absolute()))

    #removing the temporary files
    temp_folder = Path("./temp")
    temp_files = [x for x in temp_folder.glob('**/*')]
    seg_temp_files = [x for x in temp_files if str(segment_id) in str(x)]

    for f in seg_temp_files:
        f.unlink()

# ----------- 11 /11 Addition that does a last step segmentation of the soma --------- #
#return total_soma_list, run_time
#return total_soma_list_revised,run_time,total_soma_list_revised_sdf

"""
Things we should ask about the segmentation:

Advantages: 
1) could help filter away negatives

Disadvantages:
1) Can actually cut up the soma and then filter away the soma (not what we want)
2) Could introduce a big hole (don't think can guard against this)
"""


#filtered_soma_list_saved = copy.deepcopy(filtered_soma_list)

if len(filtered_soma_list) > 0:
    filtered_soma_list_revised = []
    filtered_soma_list_sdf_revised = []
    for f_soma,f_soma_sdf in zip(filtered_soma_list,filtered_soma_list_sdf):

        print("Skipping the segmentatio filter at end")
        if not (len(f_soma.faces) >= last_size_threshold and f_soma_sdf >= soma_width_threshold):
            print(f"Soma (size = {len(f_soma.faces)}, width={soma_width_threshold}) did not pass thresholds (size threshold={last_size_threshold}, width threshold = {soma_width_threshold}) ")
            continue


        if segmentation_at_end:


            if remove_inside_pieces:
                print("removing mesh interior before segmentation")
                f_soma = tu.remove_mesh_interior(f_soma,size_threshold_to_remove=size_threshold_to_remove)

            print("Doing the soma segmentation filter at end")

            meshes_split,meshes_split_sdf = tu.mesh_segmentation(
                mesh = f_soma,
                smoothness=0.5
            )
#                 print(f"meshes_split = {meshes_split}")
#                 print(f"meshes_split_sdf = {meshes_split_sdf}")

            #applying the soma width and the soma size threshold
            above_width_threshold_mask = meshes_split_sdf>=soma_width_threshold
            meshes_split_sizes = np.array([len(k.faces) for k in meshes_split])
            above_size_threshold_mask = meshes_split_sizes >= last_size_threshold

            above_width_threshold_idx = np.where(above_width_threshold_mask & above_size_threshold_mask)[0]
            if len(above_width_threshold_idx) == 0:
                print(f"No split meshes were above the width threshold ({soma_width_threshold}) and size threshold ({last_size_threshold}) so continuing")
                print(f"So just going with old somas")

                f_soma_final = f_soma
                f_soma_sdf_final = f_soma_sdf


            else:
                meshes_split = np.array(meshes_split)
                meshes_split_sdf = np.array(meshes_split_sdf)

                meshes_split_filtered = meshes_split[above_width_threshold_idx]
                meshes_split_sdf_filtered = meshes_split_sdf[above_width_threshold_idx]

                soma_width_threshold
                #way to choose the index of the top candidate
                top_candidate = 0


                largest_hole_before_seg = tu.largest_hole_length(f_soma)
                largest_hole_after_seg = tu.largest_hole_length(meshes_split_filtered[top_candidate])

                print(f"Largest hole before segmentation = {largest_hole_before_seg}, after = {largest_hole_after_seg},"
                      f"\nratio = {largest_hole_after_seg/largest_hole_before_seg}, difference = {largest_hole_after_seg - largest_hole_before_seg}")

                if largest_hole_after_seg < largest_hole_threshold:
                    f_soma_final = meshes_split_filtered[top_candidate]
                    f_soma_sdf_final = meshes_split_sdf_filtered[top_candidate]
                else:
                    f_soma_final = f_soma
                    f_soma_sdf_final = f_soma_sdf

        else:
            f_soma_final = f_soma
            f_soma_sdf_final = f_soma_sdf


        filtered_soma_list_revised.append(f_soma_final)
        filtered_soma_list_sdf_revised.append(f_soma_sdf_final)



    filtered_soma_list = np.array(filtered_soma_list_revised)
    filtered_soma_list_sdf = np.array(filtered_soma_list_sdf_revised)

return_value = list(filtered_soma_list),run_time,filtered_soma_list_sdf 

Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 5000.0 
large_mesh_threshold_inner = 5000.0 
soma_size_threshold = 937.5 
soma_size_threshold_max = 12000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
xvfb-run -n 5556 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Soma_Extraction/temp/neuron_84051.off -o /notebooks/Platinum_Datajoint/Soma_Extraction/temp/neuron_84051_fill_holes.off -s /notebooks/Platinum_Datajoint/Soma_Extraction/temp/fill_holes_383248.mls

---- meshlab output -----
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'
Current Plugins Dir is: /meshlab/src/distrib/plugins 
Error While parsing the XML filter plugin descriptors: We are trying to load a xml file that does not correspond to any dll or javascript code; please delete all the spurious xml files
Error While parsing the XML filter plugin descriptors: We are trying to load a xml file that does not correspond to any dll or 

WARNING - 2020-11-22 10:21:44,579 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:44,580 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:44,597 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:44,597 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:44,719 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:44,721 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:44,739 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:44,774 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:44,776 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:44,818 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:44,819 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:44,827 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:44,835 - base - face_normals all zero, ignoring!
WARNING - 20

WARNING - 2020-11-22 10:21:46,061 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:46,065 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:46,066 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:46,069 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:46,070 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:46,072 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:46,073 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:46,087 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:46,089 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:46,120 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:46,126 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:46,156 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:46,157 - base - face_normals all zero, ignoring!
WARNING - 20

Removing the following inside neurons: [<trimesh.Trimesh(vertices.shape=(2853, 3), faces.shape=(6331, 3))>, <trimesh.Trimesh(vertices.shape=(2721, 3), faces.shape=(6525, 3))>, <trimesh.Trimesh(vertices.shape=(1954, 3), faces.shape=(4622, 3))>, <trimesh.Trimesh(vertices.shape=(1723, 3), faces.shape=(2712, 3))>, <trimesh.Trimesh(vertices.shape=(1686, 3), faces.shape=(2629, 3))>, <trimesh.Trimesh(vertices.shape=(1623, 3), faces.shape=(2497, 3))>, <trimesh.Trimesh(vertices.shape=(1613, 3), faces.shape=(2477, 3))>, <trimesh.Trimesh(vertices.shape=(1502, 3), faces.shape=(3854, 3))>, <trimesh.Trimesh(vertices.shape=(1178, 3), faces.shape=(2712, 3))>, <trimesh.Trimesh(vertices.shape=(1122, 3), faces.shape=(2583, 3))>, <trimesh.Trimesh(vertices.shape=(998, 3), faces.shape=(2400, 3))>, <trimesh.Trimesh(vertices.shape=(894, 3), faces.shape=(2047, 3))>, <trimesh.Trimesh(vertices.shape=(824, 3), faces.shape=(1942, 3))>, <trimesh.Trimesh(vertices.shape=(757, 3), faces.shape=(1117, 3))>, <trimesh.Tri

WARNING - 2020-11-22 10:21:54,052 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:54,162 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:54,163 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:54,261 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:54,263 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:54,269 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:54,279 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:54,290 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:54,295 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:54,296 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:54,306 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:54,306 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:21:54,310 - base - face_normals all zero, ignoring!
WARNING - 20

...

mesh.is_watertight = True
/notebooks/Platinum_Datajoint/Soma_Extraction/Poisson_temp/poisson_120326.mls is being deleted....
Inside sphere validater: ratio_val = 535.8602509985581
--->This soma mesh was not added because it did not pass the sphere validation:
 soma_mesh = <trimesh.Trimesh(vertices.shape=(730, 3), faces.shape=(1453, 3))>, curr_side_len_check = True, curr_volume_check = False
There were soma pairings: Connected components in = [{0, 1}] 
Final total_soma_list_revised = [<trimesh.Trimesh(vertices.shape=(1950, 3), faces.shape=(3711, 3))>]
Final total_soma_list_revised_sdf = [0.59858]



 Total time for run = 55.93179750442505
Before Filtering the number of somas found = 1
Performing Soma Mesh Backtracking to original mesh


WARNING - 2020-11-22 10:22:25,740 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:22:25,852 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:22:25,853 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:22:25,948 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:22:25,951 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:22:25,956 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:22:25,966 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:22:25,976 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:22:25,981 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:22:25,982 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:22:25,992 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:22:25,992 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 10:22:25,996 - base - face_normals all zero, ignoring!
WARNING - 20

# total split meshes = 30
viable_meshes = [0]
There were 29 pieces found after size threshold
# of soma containing seperate meshes = 1
meshes with somas = {0: [0]}


----Working on soma-containing mesh piece 0----
current_soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(1950, 3), faces.shape=(3711, 3))>]
current_mesh = <trimesh.Trimesh(vertices.shape=(97900, 3), faces.shape=(195455, 3))>

inside Soma subtraction
mesh pieces in subtact soma BEFORE the filtering inside pieces = [<trimesh.Trimesh(vertices.shape=(32884, 3), faces.shape=(65564, 3))>, <trimesh.Trimesh(vertices.shape=(26820, 3), faces.shape=(53512, 3))>, <trimesh.Trimesh(vertices.shape=(7562, 3), faces.shape=(15047, 3))>, <trimesh.Trimesh(vertices.shape=(5781, 3), faces.shape=(11539, 3))>, <trimesh.Trimesh(vertices.shape=(5683, 3), faces.shape=(11326, 3))>, <trimesh.Trimesh(vertices.shape=(1981, 3), faces.shape=(3936, 3))>, <trimesh.Trimesh(vertices.shape=(1630, 3), faces.shape=(3235, 3))>, <trimesh.Trimesh(vertices.shape=(


Largest hole before segmentation = 53012.86141472555, after = 1164182.7847314859,
ratio = 21.96038383259401, difference = 1111169.9233167602


/meshAfterParty/trimesh_utils.py:1191: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  total_submeshes_idx =np.array(list(total_submeshes_idx.values()))
/meshAfterParty/networkx_utils.py:511: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  cycles_list_array = np.array(cycles_list)


In [27]:
largest_mesh_path_inner_decimated,dec_splits[0],largest_mesh_path_inner_decimated_clean

(<trimesh.Trimesh(vertices.shape=(4481, 3), faces.shape=(8962, 3))>,
 <trimesh.Trimesh(vertices.shape=(4481, 3), faces.shape=(8962, 3))>,
 <trimesh.Trimesh(vertices.shape=(4481, 3), faces.shape=(8962, 3))>)

In [21]:
nviz.plot_objects(meshes=[largest_mesh_path_inner_decimated],meshes_colors="random")

In [15]:
import neuron_visualizations as nviz
nviz.plot_objects(soma_mesh)

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))
